In [1]:
import pandas as pd

In [25]:
df = pd.read_csv('historical_data.csv')
print(df.head(3))

   transaction_id transaction_date transaction_time  transaction_qty store_id  \
0               1       2024-01-01         07:06:11                2      ST2   
1               2       2024-01-01         07:08:56                2      ST2   
2               3       2024-01-01         07:14:04                2      ST2   

   product_id  unit_price  
0          32         3.0  
1          57         3.1  
2          59         4.5  


In [26]:
df['transaction_date'] = pd.to_datetime(
    df['transaction_date'], format='%Y-%m-%d')

In [33]:
qunatity_sum_df = df.groupby(['store_id', 'product_id', 'transaction_date']).agg({
    "transaction_qty": "sum"}).reset_index()
qunatity_sum_df.to_csv('quantity_sum.csv', index=False)

In [35]:
quntity_sum_sorted_by_date_df = qunatity_sum_df.sort_values(['transaction_date', 'store_id', 'product_id']).reset_index(drop=True)
quntity_sum_sorted_by_date_df = quntity_sum_sorted_by_date_df.reindex(
    ['transaction_date', 'store_id', 'product_id', 'transaction_qty'], axis=1)
quntity_sum_sorted_by_date_df.to_csv('quntity_sum_sorted_by_date.csv', index=False)

In [36]:
# Create a DataFrame with all combinations of dates, store-ids, and product_ids
all_combinations = pd.DataFrame(
    index=pd.MultiIndex.from_product([
        pd.date_range(start='2024-01-01', end='2024-03-31'),
        ['ST1', 'ST2', 'ST3'],
        range(1, 88)  # product_ids from 1 to 87
    ], names=['transaction_date', 'store_id', 'product_id'])
).reset_index()

In [37]:
# Merge with the original DataFrame
quntity_sum_sorted_by_date_complete_df = pd.merge(
    all_combinations,
    quntity_sum_sorted_by_date_df,
    on=['transaction_date', 'store_id', 'product_id'],
    how='left'
)

# Fill missing values
quntity_sum_sorted_by_date_complete_df['transaction_qty'].fillna(
    0, inplace=True)
# merged_df['transaction_id'].apply(lambda x: [] if pd.isnull(x) else x)
quntity_sum_sorted_by_date_complete_df.to_csv(
    'quntity_sum_sorted_by_date_complete.csv', index=False)

In [14]:
all_combinations

,transaction_date,store_id,product_id
0,2024-01-01,ST1,1
1,2024-01-01,ST1,2
2,2024-01-01,ST1,3
3,2024-01-01,ST1,4
4,2024-01-01,ST1,5
...,...,...,...
23746,2024-03-31,ST3,83
23747,2024-03-31,ST3,84
23748,2024-03-31,ST3,85
23749,2024-03-31,ST3,86


In [3]:
# Create new feature column 'day_of_week'
df['day_of_week'] = df['transaction_date'].dt.dayofweek
df.to_csv('with_day_numbers.csv', index=False)
print(df.head(3))

   transaction_id transaction_date transaction_time  transaction_qty store_id  \
0               1       2024-01-01         07:06:11                2      ST2   
1               2       2024-01-01         07:08:56                2      ST2   
2               3       2024-01-01         07:14:04                2      ST2   

   product_id  unit_price  day_of_week  
0          32         3.0            0  
1          57         3.1            0  
2          59         4.5            0  


In [ ]:
# unique_records = df[['store_id', 'product_id']].drop_duplicates().sort_values(['store_id', 'product_id']).reset_index(drop=True)
# print(unique_records[:])
# unique_records.to_csv('sorted.csv', index=False)

    store_id  product_id
0        ST1           1
1        ST1           2
2        ST1           3
3        ST1           4
4        ST1           5
..       ...         ...
234      ST3          81
235      ST3          82
236      ST3          83
237      ST3          84
238      ST3          87

[239 rows x 2 columns]


In [57]:
# grouped_by_day = df.groupby(['store_id', 'product_id', 'transaction_date']).sum(
#     "transaction_qty")
# grouped_by_day.to_csv('grouped.csv', index=False)

In [7]:
# grouped_by_day_2 = df.groupby(['transaction_date', 'store_id', 'product_id' ]).agg({
#     "transaction_qty": "sum",
#     "transaction_id": list
# }).reset_index()

# grouped_by_day_2.to_csv('grouped_2.csv', index=False)

In [46]:
unique_dates = quntity_sum_sorted_by_date_complete_df[[
    'transaction_date']].drop_duplicates().values

print(unique_dates)
print(unique_dates.shape)
print(unique_dates.dtype)

[['2024-01-01T00:00:00.000000000']
 ['2024-01-02T00:00:00.000000000']
 ['2024-01-03T00:00:00.000000000']
 ['2024-01-04T00:00:00.000000000']
 ['2024-01-05T00:00:00.000000000']
 ['2024-01-06T00:00:00.000000000']
 ['2024-01-07T00:00:00.000000000']
 ['2024-01-08T00:00:00.000000000']
 ['2024-01-09T00:00:00.000000000']
 ['2024-01-10T00:00:00.000000000']
 ['2024-01-11T00:00:00.000000000']
 ['2024-01-12T00:00:00.000000000']
 ['2024-01-13T00:00:00.000000000']
 ['2024-01-14T00:00:00.000000000']
 ['2024-01-15T00:00:00.000000000']
 ['2024-01-16T00:00:00.000000000']
 ['2024-01-17T00:00:00.000000000']
 ['2024-01-18T00:00:00.000000000']
 ['2024-01-19T00:00:00.000000000']
 ['2024-01-20T00:00:00.000000000']
 ['2024-01-21T00:00:00.000000000']
 ['2024-01-22T00:00:00.000000000']
 ['2024-01-23T00:00:00.000000000']
 ['2024-01-24T00:00:00.000000000']
 ['2024-01-25T00:00:00.000000000']
 ['2024-01-26T00:00:00.000000000']
 ['2024-01-27T00:00:00.000000000']
 ['2024-01-28T00:00:00.000000000']
 ['2024-01-29T00:00:

In [47]:
for date in unique_dates:
    records = quntity_sum_sorted_by_date_complete_df[(quntity_sum_sorted_by_date_complete_df['transaction_date'] == date[0])]

    print(records)
    records.to_csv('record_1.csv', index=False)
    # insert data to model here
    break



    # print(store_id, product_id)



    # print(records.loc[:, ['transaction_qty']])



    # Now 'records' is a DataFrame containing all records for the current 'store_id', 'product_id' pair



    # You can now convert 'records' into a suitable format and feed it into your RNN

    transaction_date store_id  product_id  transaction_qty
0         2024-01-01      ST1           1              0.0
1         2024-01-01      ST1           2              0.0
2         2024-01-01      ST1           3              0.0
3         2024-01-01      ST1           4              0.0
4         2024-01-01      ST1           5              0.0
..               ...      ...         ...              ...
256       2024-01-01      ST3          83              0.0
257       2024-01-01      ST3          84              0.0
258       2024-01-01      ST3          85              0.0
259       2024-01-01      ST3          86              0.0
260       2024-01-01      ST3          87              3.0

[261 rows x 4 columns]
